In [1]:
import json

def filter_and_process_jsonl(input_path, output_path):
    with open(input_path, 'r', encoding="utf8") as infile, open(output_path, 'w', encoding="utf8") as outfile:
        for line in infile:
            # Parse the JSON line
            data = json.loads(line)
            
            # Check if split == "train"
            if data["split"] == "val":
                bbox = []
                for cell in data['html']['cells']:
                    if 'bbox' in cell:
                        bbox.append(cell['bbox'])
                cell_i = 0
                
                num_thead_bboxes = 0
                for token in data['html']['structure']['tokens']:
                    if token == '</thead>':
                        break
                    if token in ['<td>', '<td']:
                        cell = data['html']['cells'][cell_i]
                        if 'bbox' in cell:
                            num_thead_bboxes += 1
                        cell_i += 1
                category_id = ([0] * num_thead_bboxes) + ([1] * (len(bbox) - num_thead_bboxes))

                annotations = []
                for b, c in zip(bbox, category_id):
                    xmin, ymin, xmax, ymax = b
                    width = xmax - xmin
                    height = ymax - ymin
                    area = width * height
                    annotation = {
                        'bbox': [xmin, ymin, width, height],
                        'category_id': c,
                        'area': area
                    }
                    annotations.append(annotation)

                output = {
                    'filename': data['filename'],
                    'image_id': data['imgid'],
                    'html': data['html']['structure']['tokens'],
                    'annotations': annotations
                }

                # Write the output JSON object to the file with a newline character
                outfile.write(json.dumps(output) + '\n')

# Example usage
input_jsonl_path = r"C:\Users\tangy\Downloads\pubtabnet\pubtabnet\PubTabNet_2.0.0.jsonl"
output_jsonl_path = r"C:\Users\tangy\Downloads\pubtabnet\pubtabnet\val.jsonl"
input_jsonl_path = r"C:\Users\remote desktop\Downloads\unitable-main\dataset\mini_pubtabnet\mini_pubtabnet_examples.jsonl"
output_jsonl_path = r"testval.jsonl"

filter_and_process_jsonl(input_jsonl_path, output_jsonl_path)